In [1]:
import pickle
with open('../data/data.pickle', 'rb') as f:
  data = pickle.load(f)

In [2]:
my_stop_words = ['']
texts = data['extrawordsplit'].tolist()
type(texts)

list

In [3]:
def flat(l):
    for k in l:
        if not isinstance(k, (list, tuple)):
            yield k
        else:
            yield from flat(k)
texts = list(flat(texts))

First step: Deal with password without capital words

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# change the max_df to get different results
tfidf_vectorizer = TfidfVectorizer(max_df=0.9999, max_features=20000,
                                   min_df=0.0001, stop_words=my_stop_words, 
                                   use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print(tfidf_matrix.shape)


CPU times: user 361 ms, sys: 4.25 ms, total: 366 ms
Wall time: 364 ms
(108672, 1257)


In [9]:
len(data)

38650

In [10]:
terms = tfidf_vectorizer.get_feature_names()

In [11]:
len(terms)

1257

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [12]:
from sklearn.cluster import KMeans
# num of clusters depends on dataset 
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 17.7 s, sys: 27.9 ms, total: 17.7 s
Wall time: 2.97 s


In [14]:
import pandas as pd
df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray())


In [19]:
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')

Top terms per cluster:

Cluster 0 words:

NameError: name 'vocab_frame' is not defined